In [ ]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

In [ ]:
# Import necessary libraries
from qiskit import QuantumCircuit, Aer, execute
from qiskit.quantum_info import Operator
import numpy as np
from qiskit.visualization import plot_histogram

In [ ]:
# Define a phase oracle
def phase_oracle(n, indices_to_mark, name='Oracle'):
    # Create a quantum circuit on n qubits
    qc = QuantumCircuit(n, name=name)
    
    # Create the identity matrix on n qubits
    oracle_matrix = np.identity(2**n)
    
    # Add the -1 phase to marked elements
    for index_to_mark in indices_to_mark:
        oracle_matrix[index_to_mark, index_to_mark] = -1

    # Convert the matrix (called oracle_matrix) into an operator and add it to the quantum circuit
    qc.unitary(Operator(oracle_matrix), range(n))
    
    return qc


In [ ]:
# Define a diffuser
def diffuser(n):
    # Create a quantum circuit on n qubits
    qc = QuantumCircuit(n, name='Diffuser')
      
    # Apply Hadamard gates to all qubits
    qc.h(range(n))
    
    # Call the phase oracle applied to the zero state
    qc.append(phase_oracle(n, [0]), range(n))
    
    # Apply Hadamard gates to all qubits
    qc.h(range(n))
     
    return qc

In [ ]:
# Define the Grover algorithm
def Grover(n, indices_of_marked_elements):
    # Create a quantum circuit on n qubits
    qc = QuantumCircuit(n, n)
    
    # Determine r (number of iterations)
    r = int(np.floor(np.pi/4*np.sqrt(2**n/len(indices_of_marked_elements))))
    print(f'{n} qubits, basis states {indices_of_marked_elements} marked, {r} rounds')
    
    # Step 1: Apply Hadamard gates to all qubits
    qc.h(range(n))
    
    # Step 2: Apply r rounds of the phase oracle and the diffuser
    for _ in range(r):
        qc.append(phase_oracle(n, indices_of_marked_elements), range(n))
        qc.append(diffuser(n), range(n))
        
    # Step 3: Measure all qubits
    qc.measure(range(n), range(n))
  
    return qc

In [ ]:
# Create a Grover circuit with 6 qubits and marked elements at [1, 42]
mycircuit = Grover(6, [1, 42])


In [ ]:
# Draw the circuit
mycircuit.draw()


In [ ]:
# Choose a simulator backend
simulator = Aer.get_backend('qasm_simulator')


In [ ]:
# Execute the circuit and get the results
counts = execute(mycircuit, backend=simulator, shots=1000).result().get_counts(mycircuit)


In [ ]:
# Plot the histogram of measurement results
plot_histogram(counts)